In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

## Setup

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
}

data_dir = 'data'

image_datasets = {x: datasets.ImageFolder(os.path.join(
    data_dir, x), data_transforms[x]) for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0

    # Keep track of training history so we can report on overfitting, etc.
    hist = {
        "running_loss": [],
        "running_acc": [],
        "val_loss": [],
        "val_acc": []
    }

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)


        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == "train":
                hist['running_loss'].append(epoch_loss)
                hist['running_acc'].append(epoch_acc)
            elif phase == "val":
                hist['val_loss'].append(epoch_loss)
                hist['val_acc'].append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_epoch = epoch

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('Best model occurred after ' + str(best_epoch) + ' epochs.')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, hist

## Run Model

In [4]:
# Load pretrained resnet18 model
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Set number of output features to 3 ("2_2", "3_4", "4_4")
model_ft.fc = nn.Linear(num_ftrs, 3)

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model_ft, hist = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                    num_epochs=25)

Epoch 0/24
----------
train Loss: 0.8973 Acc: 0.6233
val Loss: 0.2563 Acc: 0.9010

Epoch 1/24
----------
train Loss: 0.8050 Acc: 0.6956
val Loss: 0.0735 Acc: 0.9670

Epoch 2/24
----------
train Loss: 0.7184 Acc: 0.7333
val Loss: 0.1114 Acc: 0.9670

Epoch 3/24
----------
train Loss: 0.6513 Acc: 0.7456
val Loss: 0.0567 Acc: 0.9967

Epoch 4/24
----------
train Loss: 0.5815 Acc: 0.7556
val Loss: 0.1477 Acc: 0.9274

Epoch 5/24
----------
train Loss: 0.5649 Acc: 0.7911
val Loss: 0.0852 Acc: 0.9901

Epoch 6/24
----------
train Loss: 0.5922 Acc: 0.7622
val Loss: 0.1752 Acc: 0.9340

Epoch 7/24
----------
train Loss: 0.4518 Acc: 0.8267
val Loss: 0.0250 Acc: 1.0000

Epoch 8/24
----------
train Loss: 0.4303 Acc: 0.8456
val Loss: 0.0282 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.4784 Acc: 0.8211
val Loss: 0.0416 Acc: 0.9967

Epoch 10/24
----------
train Loss: 0.4218 Acc: 0.8422
val Loss: 0.0398 Acc: 0.9967

Epoch 11/24
----------
train Loss: 0.4533 Acc: 0.8378
val Loss: 0.0298 Acc: 1.0000

Ep

In [5]:
torch.save(model_ft.state_dict(), 'ct_model3.pt')